In [ ]:
class FAA:
    #small functions are named lowercase__lowercase__lowercase
    #Important functions are named Capitilized+lower__lower__lower  
    def __init__(self,backtest_Nyears=20,N=3,Universe=['SPY','SHY'],cash_deposit='SHY',rebalance_month=1,lookback_period=4):
        try:
            import numpy as np
        except:
            print('requred library: Numpy')
        try:
            import matplotlib.pyplot as plt
            print('libraries well imported.')
        except:
            print('requred library: matplotlib')
        
        
        self.lookback_period = lookback_period*24
        #Get rid of lowercase letters for the sorting process afterwards. 
        if cash_deposit not in Universe:
            Universe.append(cash_deposit)
        self.Universe = '_'.join(sorted(Universe,key=str.lower)).upper().split('_')
        self.backtest_Nyears = backtest_Nyears
#        self.backtest_from = backtest_from
        self.rebalancing_period = rebalance_month
        
        self.trading_cost = 0.005
        self.N = N
        self.portfolio = {keys:0 for keys in self.Universe}
        self.cash_deposit = cash_deposit
        #self.value는 성과 추적을 위함.
        self.cash = 100_0000
        self.value = self.cash
        self.invested = []
        self.initial_value = self.cash

        self.Load_data()
        self.value_log_dict = {}
        self.upgraded_vol_template = pd.Series(data={ticker:0 for ticker in self.Universe},index=self.Universe)

    def Load_data(self,):
        qb = QuantBook()
        for ticker in self.Universe:
            qb.AddEquity(ticker,Resolution.Daily)
        history = qb.History(qb.Securities.Keys,timedelta(365*self.backtest_Nyears), Resolution.Daily)
        # 종가로 정렬
        df = history['close'].unstack(level=0)
        df.columns = self.Universe
        self.df = df.iloc[:]
        self.last_day = df.index[-1]
        #계좌 기록용 df
        self.account_df = pd.DataFrame([[0 for _ in self.Universe]],[df.index[0]],self.Universe)
        self.account_df['cash'] = self.cash



    def roll_months(self,month,rebalancing_period=1):
        months = [1,2,3,4,5,6,7,8,9,10,11,12]
        return months[(month+rebalancing_period)%12-1]

    def next_rebalancing_date(self,date,rebalancing_period=1):
        if date.month+rebalancing_period > 12:
            NRyear = date.year + (date.month+rebalancing_period)//12
        else:
            NRyear = date.year
        NRmonth = self.roll_months(date.month,rebalancing_period=self.rebalancing_period)

        #데이터의 마지막 달을 넘어서면 False반환
        if datetime(NRyear,NRmonth,1) > self.last_day:
            return False

        #그 다음달 첫 거래일
        next_trading_day = self.df.loc[f'{NRyear}-{NRmonth}'].iloc[0].name
        return next_trading_day

    
    def whats_invested(self,):
        lst = []
        for ticker in self.Universe:
            if self.portfolio[ticker]>0:
                lst.append(ticker)
        self.invested = lst


    def change_position(self,ticker='SPY',quantity=0,quantity_in_dollars=0):
        price = self.today_row[ticker]
        if quantity > 0:
            #현재 보유한 cash*quantity만큼 매수    
            if quantity <= 1:
                budget = self.cash*quantity
                count = budget//(price*(1+self.trading_cost))
                if count > 0:
                    self.cash -= price*count*(1+self.trading_cost)
                    self.portfolio[ticker] += count
        elif quantity < 0:
            #보유_주식_수*quantity를 정수가 되도록 버림한 수만큼 매도
            if quantity >= -1:
                count = int(price*quantity*(1-self.trading_cost))
                if count > 0:
                    self.cash += price*count*(1-self.trading_cost)
                    self.portfolio[ticker] -= count
        elif quantity_in_dollars > 0:
                budget = self.cash*quantity
                count = budget//(price*(1+self.trading_cost))
                if count > 0:
                    self.cash -= price*count*(1+self.trading_cost)
                    self.portfolio[ticker] += count
        elif quantity_in_dollars < 0:
                count = int(price*quantity*(1-self.trading_cost))
                if count > 0:
                    self.cash += price*count*(1-self.trading_cost)
                    self.portfolio[ticker] -= count
        self.whats_invested()
    

    def liquidate(self,ticker):
        if ticker in self.invested:
            self.cash += self.trading_row[ticker]*self.portfolio[ticker]*(1-self.trading_cost)
            self.portfolio[ticker]=0
            self.whats_invested()

    def Play(self,weight_vector=(1,1,1)):
        self.cash = self.initial_value
        #weight vector unpacked in the order of 'return','volatility','corrrelation'
        self.weight_vector = weight_vector

        dates = self.df.index.to_series()
        dates.index = pd.RangeIndex(len(dates))

        #데이터가 준비된 날 바로 다음 달 거래를 시작함.
        first_ready_day =  dates.iloc[self.lookback_period]
        trading_day = self.next_rebalancing_date(first_ready_day,rebalancing_period=1)
        #print(trading_day)

        while True:
            day_count = int(dates.index[dates==trading_day].to_list()[0])
            loss_dict = self.Loss(day_count)
            loss_list = pd.DataFrame.from_dict(data=loss_dict, orient='index',columns=['Loss']).sort_values(by='Loss').index.to_list()
            to_buy = loss_list[:self.N]

            self.today_row = self.df.iloc[day_count]
            #return이 0보다 작으면 탈락
            for ticker in to_buy:
                if not self.overzero[ticker]:
                    to_buy.remove(ticker)

            self.trading_row = self.df.loc[trading_day]

            #정리해야할 자산들 청산.
            for ticker in self.invested:
                if ticker not in to_buy:
                    self.liquidate(ticker)



            #사야할 자산은 매수.
            for ticker in to_buy:
                if ticker not in self.invested:
                    self.change_position(ticker=ticker,quantity=1/len(to_buy))
#                    self.change_position(ticker=ticker,quantity=1/self.N)
            if self.overzero[to_buy[0]]:
                self.change_position(ticker=to_buy[0],quantity=1)

            #남은 돈은 현금성자산에 투자
            if self.overzero[self.cash_deposit] and self.cash_deposit in to_buy:
                self.change_position(ticker=to_buy[0],quantity=1)
                #self.change_position(ticker=self.cash_deposit,quantity=1)

            current_value_in_dollar = [self.trading_row[ticker]*self.portfolio[ticker] for ticker in self.trading_row.index]
            self.account_df.loc[trading_day] = current_value_in_dollar+[self.cash,]

            #계좌 총 평가 금액 계산
            holdings_count = [self.portfolio[ticker] for ticker in self.trading_row.index]
            self.value = self.cash + self.trading_row@holdings_count

            #다음 거래 일자 갱신
            trading_day = self.next_rebalancing_date(trading_day,rebalancing_period=self.rebalancing_period)

            #다음 거래일이 데이터 마지막 날보다 미래면 모두 청산하고 종료
            if trading_day == False:
                for ticker in self.invested:
                    self.liquidate(ticker)
                #계좌 최종 평가 금액 계산
                self.value = self.cash
                break

    def Loss(self,day=0):
        self.lr = self.ret(day)
        self.lv = self.upgraded_vol(day)
        #self.lv = self.vol(day)
        self.lc = self.cor(day)

        loss_dict = {}
        for ticker in self.Universe:
            #r,v,c = return self.lr.index(ticker), self.lv.index(ticker), self.lc.index(ticker)
            r,v,c = self.lr.index(ticker)+1, self.lv.index(ticker)+1, self.lc.index(ticker)+1
            wr,wv,wc = self.weight_vector   
            loss_dict[ticker] = wr*r + wv*v + wc*c
        return loss_dict

    # ['SPY','SHY'] 형태로 순서에 따라 정렬된 ticker의 list를 구함. (각각 lr, lc, lv)
    def ret(self,day):
        r = self.df.iloc[day-1]/self.df.iloc[day-self.lookback_period]-1
        self.overzero = ( r>0 )
        #return이 작을수록 rank가 커짐
        r = r.sort_values(ascending=False)
        return r.index.to_list()

    def vol(self,day):
        vol = self.df.iloc[day-self.lookback_period:day].std()
        #vol이 클수록 rank가 커짐
        return vol.sort_values(ascending=True).index.to_list()

    def cor(self,day):
        corr_sum = self.df.iloc[day-self.lookback_period:day].corr(method='pearson').sum()
        #corr이 클수록 rank가 커짐
        return corr_sum.sort_values(ascending=True).index.to_list()

    def upgraded_vol(self,day):
        d = self.df.iloc[day-self.lookback_period:day]
        adjd = d - (np.matrix(np.arange(1,len(d)+1)).T @ np.matrix((d.iloc[-1]-d.iloc[0]).to_numpy())) - d.iloc[0]
        adjd = adjd.std()
        return adjd.sort_values(ascending=True).index.to_list()

    def real_vol(self,xs):
        #직선 추세를 제외한 변동성을 산출.
        adjxs = xs-np.arange(1,len(xs)+1)*((xs[-1]-xs[0])/len(xs))-xs[0]
        return adjxs.std()

    def Analyze(self,plot=True,prnt=True):
        adf = self.account_df
        net = adf.sum(axis=1)
        annual_return = ((net[-1]/self.initial_value)**(1/self.backtest_Nyears)-1)*100


        real_vol = self.real_vol(net[1:])
        if plot:
            plt.plot(  net[1:]- (np.arange(1,len(net))*((net[-1]-net[1])/(len(net)-1)))-net[1])
            plt.plot((net.index[1],net.index[-1]),(0,0))
            
            plt.plot(net[1:])
            plt.plot((net.index[1],net.index[-1]),(net[1],net[-1]))  
            plt.show()      
        if prnt:
            print(f'{annual_return=}')
            print('real_vol_score:', real_vol, '\nrelative_vol:', real_vol/annual_return/self.initial_value*1000)
        else:
            return annual_return,real_vol,real_vol/annual_return/self.initial_value*1000





U = ['IWM','EFA','VV','TLT','SHY','IEF','VNQ','SCHH','IYR','GLD','GSG']
faa = FAA(Universe=U,cash_deposit='SHY',rebalance_month=3,N=4,lookback_period=4)
if 0:
    faa.Play(weight_vector=(10,2,4))
    faa.Analyze(plot=1,prnt=1)
    #faa.Play(weight_vector=(4,0,6)) good!! N=4
    faa.Play(weight_vector=(4,0,6))
    faa.Analyze(plot=1,prnt=1)
    faa.Play(weight_vector=(1,0,2.4))
    faa.Analyze(plot=1,prnt=1)
    faa.Play(weight_vector=(1,0,0.5))
    faa.Analyze(plot=1,prnt=1)


faa.Play(weight_vector=(8,-2,2))
faa.Analyze(plot=1,prnt=1)
faa.Play(weight_vector=(8,2,2))
faa.Analyze(plot=1,prnt=1)


relative_vol: the smaller the better. around 40 is very good. over 70 can is very volatile.

![Alt text](8,-2,2.png)

![Alt text](8,2,2.png)